# 最大熵原理
## 凡是我们知道的，把它考虑进去，凡是不知道的，通通均匀分布
$$
H(P) = - \sum P(x) \log {P(x)} \\
$$

## 模型
$$
\max H(P) = - \sum_{x, y} \tilde{P}(x) P(y | x) \log P(y | x)
$$

In [3]:
import numpy as np
import math
from copy import deepcopy

In [8]:
# 最大熵
class MaxEntropy:
    def __init__(self, EPS = 0.005):
        self._samples = []
        self._Y = set() # 标签集合
        self._numXY = {}  # key为(x, y)， value为次数
        self._N = 0 # 样本数
        self._Ep_ = [] # 样本分布的特征期望值
        self._xyID = {} #key记录(x, y), value 记录 id号
        self._n = 0 # 特征键值(x, y)的个数
        self._C = 0 # 最大特征数
        self._IDxy = {} # ke为(x, y) vaule 为对应的id号
        self._w = []
        self._EPS = EPS #收敛条件
        self._lastw = [] # 上一次w参数值
        
    def load_data(self, dataset):
        self._samples = deepcopy(dataset)
        for items in self._samples:
            y = items[0]
            X = items[1: ]
            self._Y.add(y)  # 集合中y若已存在则会自动忽略
            for x in X:
                if (x, y) in self._numXY:
                    self._numXY[(x, y)] += 1
                else:
                    self._numXY[(x, y)] = 1
            
        self._N = len(self._samples)
        self._n = len(self._numXY)
        self._C = max([len(sample) - 1 for sample in self._samples])
        self._w = [0] * self._n
        self._lastw = self._w[:]
        
        self._Ep_ = [0] * self._n
        for i, xy in enumerate(self._numXY):
            self._Ep_[i] = self._numXY[xy] / self._N
            self._xyID[xy] = i
            self._IDxy[i] = xy
            
    
    def _Zx(self, X):
        """
        计算Z(x)的值
        """
        zx = 0
        for y in self._Y:
            ss = 0
            for x in X:
                if (x, y) in self._numXY:
                    ss += self._w[self._xyID[(x, y)]]
            zx += math.exp(ss)
        return zx
    
    def _model_pyx(self, y, X):
        """
        计算P(y|x)
        """
        zx = self._Zx(X)
        ss = 0
        for x in X:
            if (x, y) in self._numXY:
                ss += self._w[self._xyID[(x, y)]]
        pyx = math.exp(ss) / zx
        return pyx
    
    def _model_ep(self, index):
        """
        计算特征函数fi关于模型的期望
        """
        x, y = self._IDxy[index]
        ep = 0
        for sample in self._samples:
            if x not in sample:
                continue
            pyx = self._model_pyx(y, sample)
            ep += pyx / self._N
        return ep
    
    def _convergenc(self):
        """
        判断是否收敛
        """
        for last, now in zip(self._lastw, self._w):
            if abs(last - now) >= self._EPS:
                return False
        return True
    
    def predict(self, X):
        """
        计算预测概率
        """
        Z = self._Zx(X)
        result = {}
        for y in self._Y:
            ss = 0
            for x in X:
                if (x, y) in self._numXY:
                    ss += self._w[self._xyID[(x, y)]]
            pyx = math.exp(ss) / Z
            result[y] = pyx
        return result
    
    def train(self, maxiter = 100):
        """
        训练数据
        """
        for loop in range(maxiter):
            print("iter: %d" % loop)
            self._lastw = self._w[ : ]
            for i in range(self._n):
                ep = self._model_ep(i)
                self._w[i] += math.log(self._Ep_[i] / ep) / self._C
            print("w: ", self._w)
            
            if self._convergenc():
                break
                
        

In [5]:
dataset = [['no', 'sunny', 'hot', 'high', 'FALSE'],
            ['no', 'sunny', 'hot', 'high', 'TRUE'],
            ['yes', 'overcast', 'hot', 'high', 'FALSE'],
            ['yes', 'rainy', 'mild', 'high', 'FALSE'],
            ['yes', 'rainy', 'cool', 'normal', 'FALSE'],
            ['no', 'rainy', 'cool', 'normal', 'TRUE'],
            ['yes', 'overcast', 'cool', 'normal', 'TRUE'],
            ['no', 'sunny', 'mild', 'high', 'FALSE'],
            ['yes', 'sunny', 'cool', 'normal', 'FALSE'],
            ['yes', 'rainy', 'mild', 'normal', 'FALSE'],
            ['yes', 'sunny', 'mild', 'normal', 'TRUE'],
            ['yes', 'overcast', 'mild', 'high', 'TRUE'],
            ['yes', 'overcast', 'mild', 'high', 'TRUE'],
            ['no', 'rainy', 'mild', 'high', 'TRUE']]

In [9]:
maxnet = MaxEntropy()
x = ['overcast', 'mild', 'high', 'FALSE']


In [10]:
maxnet.load_data(dataset)
maxnet.train()

iter: 0
w:  [0.0455803891984887, 0.06815136616784778, -0.005270858269468233, -0.1443607729138983, -0.040991916953456525, 0.16665885939351632, -0.11018096671854669, -0.009355786916112944, 0.07754277284698692, 0.027806333368878385, 0.06975549747250427, 0.08039685194073605, 0.10013111159389536, -0.014202283299991545, -0.12383526657979044, -0.21622698021385453, -0.0025765175096604588, -0.09601705132537947, -0.08580796448821491]
iter: 1
w:  [0.11370940103458554, 0.12809456959126594, 0.012501883343189756, -0.2361785562962871, -0.03897268889765423, 0.29436373909141683, -0.20228464875062457, -0.030510337133163833, 0.1194714620873893, 0.008237727656763543, 0.1108592046524991, 0.1053085885615596, 0.1535960510040914, 0.0257050879340702, -0.17648718721702963, -0.3732686912380385, -0.017690620764503697, -0.16545110078652775, -0.16839925560170144]
iter: 2
w:  [0.17882335104309632, 0.17026384862808153, 0.03244190076870465, -0.3066785479168635, -0.022131844039305116, 0.4046635702681687, -0.27245667285

In [11]:
print("predict: ", maxnet.predict(x))

predict:  {'no': 0.001027642500279428, 'yes': 0.9989723574997207}
